## Identify 10 Most Important Features

In [ ]:
source('../src/util.r')
source('../src/multiplot.r')

In [ ]:
ames_housing_df <- loadCleanAmesData()

In [ ]:
features_df <- ames_housing_df[, !names(ames_housing_df) %in% c("SalePrice")]

In [ ]:
dim(features_df)

In [ ]:
head(features_df)

#### Let's count the number of 0s in the columns. 

In [ ]:
zero_sums <- colSums(features_df == 0)
zero_sums <- zero_sums[zero_sums > 0]
#zero_sums[zero_sums < 1460 * 0.50]
zero_sums

#### Zero's in these columns indicate that the feature is not available in those homes. If a feature is not available in more than 50% of the homes, we can assume the features are not likely to affect the SalePrice prediction and hence remove those features from consideration.

In [ ]:
features_df <- features_df[, colSums(features_df == 0) < (1460 * 0.50)]

In [ ]:
dim(features_df)

#### If PoolArea is irrelevant as above, Pool Quality is irrelevant for our prediction and can be removed as well.

In [ ]:
features_df <- features_df[, !names(features_df) %in% c("PoolQC")]

In [ ]:
dim(features_df)

#### Split numerical and categorical features

In [ ]:
numerical_features <- Filter(is.numeric, features_df)
categorical_features <- Filter(is.factor, features_df)

In [ ]:
dim(numerical_features)
dim(categorical_features)

#### Log and scale the numerical features

In [ ]:
numerical_features = log(numerical_features + 1)

In [ ]:
numerical_features = data.frame(scale(numerical_features))

#### One Hot Encode Categorical features

In [ ]:
install.packages("onehot")

In [ ]:
library("onehot")

In [ ]:
one_hot_encoding = onehot(categorical_features, stringsAsFactors = FALSE, addNA = FALSE, max_levels = 25) 
encoded_categorical_features <- predict(one_hot_encoding, categorical_features)

#### Join the numerical features and one hot encoded categorical features

In [ ]:
rownames(numerical_features) <- ames_housing_df$Id 
rownames(encoded_categorical_features) <- ames_housing_df$Id 

In [ ]:
all_features <- merge(numerical_features, encoded_categorical_features, by=0, all=FALSE)

In [ ]:
all_features$Row.names <- NULL
all_features$Id <- NULL

In [ ]:
dim(all_features)

In [ ]:
head(all_features)

#### Let's find the correlation between features and target and choose features with the highest correlation to the target Sale price

In [ ]:
all_features_target <- all_features
all_features_target$SalePrice <- ames_housing_df$SalePrice

In [ ]:
head(all_features)

In [ ]:
install.packages("mlbench")

In [ ]:
install.packages("caret")

In [ ]:
set.seed(7)
# load the library
library(mlbench)
library(caret)

In [ ]:
#install.packages("corrplot")

In [ ]:
#library("Hmisc")
#library("corrplot")

In [ ]:
#corr_result <- rcorr(as.matrix(all_features))
#head(corr_result)

**corr_result contains correlation coefficients and correlation p-values**

#### Let's plot the significant correlations (p > 0.5) and leave the insignificant correlations blank

In [ ]:
# Insignificant correlations are leaved blank
corrplot(corr_result$r, type="upper", order="hclust", 
         p.mat = corr_result$P, sig.level = 0.50, insig = "blank")